In [ ]:
# -*- coding: utf-8 -*-
import os, re, datetime, itertools, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# =========================
# パス設定（必要に応じて変更）
# =========================
# あなたのリポジトリ構成に合わせた既定値
PROCESS_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\process"

# 使いたい軌跡サブフォルダ（例：正規化図のみ使うなら *_nor を指定）
# 例) ["tail_set_nor", "left_paw_nor", "right_paw_nor"]
# 例) 生の軌跡なら ["tail_set", "left_paw", "right_paw"]
SRC_SUBDIRS = [ 
    "left_paw_nor", 
    # "left_tarsal_nor", 
    "right_paw_nor", 
    # "right_tarsal_nor"
    ]  # ←必要に応じて変更

# 出力先
MODEL_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\models_cnn_traj"
CURVE_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\output\learning_curve_traj"
EVAL_DIR  = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\output\learning_curve_traj"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CURVE_DIR, exist_ok=True)
os.makedirs(EVAL_DIR,  exist_ok=True)

# =========================
# ハイパパラメータ
# =========================
IMG_SIZE = (300, 300)
BATCH_SIZE = 16
EPOCHS = 200
VAL_RATIO = 0.15
LEARNING_RATE = 5e-6
AUGMENT = True  # 軽いDataAugを入れる場合は True

# =========================
# ラベル推定（ファイル名/親フォルダ名から）
# =========================
CLASSES = ["normal", "ivdd"]  # 0: normal, 1: ivdd
NAME2IDX = {"normal": 0, "ivdd": 1}

def infer_label_from_path(p: Path):
    """
    one / two / ivdd -> ivdd, normal -> normal
    ファイル名と親フォルダ名をトークン化して判定
    """
    def tokens(s):
        return [t for t in re.split(r"[^a-z0-9]+", s.lower()) if t]

    all_tokens = set(tokens(p.stem)) | set(tokens(p.name)) | set(tokens(p.parent.name))
    if ("normal" in all_tokens):
        return NAME2IDX["normal"]
    if ("ivdd" in all_tokens) or ("one" in all_tokens) or ("two" in all_tokens):
        return NAME2IDX["ivdd"]
    return None  # 判定不能

# =========================
# 画像リスト収集
# =========================
def collect_images(process_dir: str, subdirs=None, exts=(".png", ".jpg", ".jpeg")):
    proc = Path(process_dir)
    paths, labels = [], []
    subdirs = list(subdirs) if subdirs else []

    if subdirs:
        cand_dirs = [proc / sd for sd in subdirs]
    else:
        # subdirs未指定なら process 直下のディレクトリすべて
        cand_dirs = [d for d in proc.iterdir() if d.is_dir()]

    for d in cand_dirs:
        if not d.exists():
            print(f"[WARN] ディレクトリが見つかりません: {d}")
            continue
        for p in d.rglob("*"):
            if p.is_file() and p.suffix.lower() in exts:
                y = infer_label_from_path(p)
                if y is None:
                    print(f"[SKIP] ラベル不明: {p}")
                    continue
                paths.append(str(p))
                labels.append(y)

    if not paths:
        raise FileNotFoundError("画像が見つかりませんでした。SRC_SUBDIRS とファイル名ルール（one/two/ivdd/normal）を確認してください。")

    df = pd.DataFrame({"path": paths, "y": labels})
    return df

df_all = collect_images(PROCESS_DIR, SRC_SUBDIRS)
print("[INFO] 収集画像数:", len(df_all))
print(df_all["y"].value_counts().rename(index={0:"normal",1:"ivdd"}))

# =========================
# Stratified Train/Val 分割
# =========================
sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_RATIO, random_state=42)
train_idx, val_idx = next(sss.split(df_all["path"], df_all["y"]))
df_tr = df_all.iloc[train_idx].reset_index(drop=True)
df_va = df_all.iloc[val_idx].reset_index(drop=True)
print(f"[INFO] split -> train: {len(df_tr)}, val: {len(df_va)}")

# =========================
# tf.data パイプライン
# =========================
AUTOTUNE = tf.data.AUTOTUNE

def decode_img(path):
    img_bytes = tf.io.read_file(path)
    # PNG/JPEG いずれも3chでデコード
    img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.clip_by_value(img/255.0, 0.0, 1.0)
    return img

def aug(img):
    # 軽いDataAug（必要に応じて調整）
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.05)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
    return img

def make_ds(paths, labels, training=True, batch_size=BATCH_SIZE):
    ds_paths = tf.data.Dataset.from_tensor_slices((paths, labels))
    def _load(path, y):
        img = decode_img(path)
        if training and AUGMENT:
            img = aug(img)
        y_onehot = tf.one_hot(y, depth=len(CLASSES))
        return img, y_onehot
    ds = ds_paths.map(_load, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(buffer_size=min(1000, len(paths)))
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(df_tr["path"].values, df_tr["y"].values, training=True)
val_ds   = make_ds(df_va["path"].values, df_va["y"].values, training=False)

# =========================
# class_weight（不均衡対策）
# =========================
cls_w = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1]),
    y=df_tr["y"].values
)
class_weight = {0: float(cls_w[0]), 1: float(cls_w[1])}
print("[INFO] class_weight:", class_weight)

# =========================
# CNN モデル（提示の構成を踏襲）
# =========================
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')  # 2クラス
])
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# =========================
#  学習
# =========================
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = os.path.join(MODEL_DIR, f"traj_cnn_{now}.h5")
curve_path = os.path.join(CURVE_DIR, f"traj_learning_curve_{now}.png")
pred_csv   = os.path.join(EVAL_DIR,  f"traj_val_predictions_{now}.csv")
cm_png     = os.path.join(EVAL_DIR,  f"traj_cm_{now}.png")

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min", factor=0.5, patience=5, min_lr=1e-6, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor="val_accuracy", mode="max", save_best_only=True, verbose=1
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight,
    verbose=1,
    callbacks=callbacks
)

print(f"✅ モデル保存: {model_path}")

# =========================
# 学習曲線保存
# =========================
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss"); plt.legend(); plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy"); plt.legend(); plt.grid(True)

plt.tight_layout()
plt.savefig(curve_path, dpi=150)
plt.close()
print(f"✅ 学習曲線保存: {curve_path}")

# =========================
#  検証データでの詳細評価（CSV & 混同行列）
# =========================
# バッチ毎に path を持ち回すため簡便に numpy で回す
val_paths = df_va["path"].values
val_labels = df_va["y"].values
y_true = []
y_prob = []
y_pred = []

for start in range(0, len(val_paths), BATCH_SIZE):
    batch_paths = val_paths[start:start+BATCH_SIZE]
    batch_imgs = np.stack([tf.image.resize(tf.io.decode_image(tf.io.read_file(p), channels=3, expand_animations=False), IMG_SIZE).numpy()/255.0
                           for p in batch_paths], axis=0)
    probs = model.predict(batch_imgs, verbose=0)
    preds = probs.argmax(axis=1)
    y_prob.append(probs)
    y_pred.append(preds)
    y_true.append(val_labels[start:start+BATCH_SIZE])

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
y_prob = np.concatenate(y_prob)

print("\n[Validation] classification_report")
print(classification_report(y_true, y_pred, target_names=CLASSES, digits=4))

# 予測CSV
df_pred = pd.DataFrame({
    "path": val_paths,
    "true": [CLASSES[i] for i in y_true],
    "pred": [CLASSES[i] for i in y_pred],
    "p_normal": y_prob[:, 0],
    "p_ivdd": y_prob[:, 1],
})
df_pred.to_csv(pred_csv, index=False, encoding="utf-8-sig")
print(f"✅ 予測CSV保存: {pred_csv}")

# 混同行列PNG
cm = confusion_matrix(y_true, y_pred, labels=[0,1])
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(cm.astype(np.float32) / max(cm.sum(),1), cmap="Blues")
ax.figure.colorbar(im, ax=ax)
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(CLASSES); ax.set_yticklabels(CLASSES)
ax.set_xlabel("Predicted"); ax.set_ylabel("True")
ax.set_title("Confusion Matrix (Validation)")
for i in range(2):
    for j in range(2):
        ax.text(j, i, int(cm[i,j]),
                ha="center",
                color="white" if cm[i,j] > cm.max()/2 else "black",
                fontsize=14)
plt.tight_layout()
plt.savefig(cm_png, dpi=150); plt.close()
print(f"✅ 混同行列保存: {cm_png}")


[INFO] 収集画像数: 2422
y
normal    1494
ivdd       928
Name: count, dtype: int64
[INFO] split -> train: 2058, val: 364
[INFO] class_weight: {0: 0.8108747044917257, 1: 1.3041825095057034}


c:\kanno\envs\MLya\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 298, 298, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │    20,070,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,164,034 (76.92 MB)

...

129/129 ━━━━━━━━━━━━━━━━━━━━ 23s 173ms/step - accuracy: 0.8898 - loss: 0.2863 - val_accuracy: 0.8874 - val_loss: 0.2689 - learning_rate: 2.5000e-06
Epoch 98/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.8996 - loss: 0.2736
Epoch 98: val_accuracy did not improve from 0.88736
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 169ms/step - accuracy: 0.8995 - loss: 0.2737 - val_accuracy: 0.8819 - val_loss: 0.2749 - learning_rate: 2.5000e-06
Epoch 99/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.8926 - loss: 0.2713
Epoch 99: val_accuracy did not improve from 0.88736
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 167ms/step - accuracy: 0.8925 - loss: 0.2714 - val_accuracy: 0.8819 - val_loss: 0.2712 - learning_rate: 2.5000e-06
Epoch 100/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.9043 - loss: 0.2643
Epoch 100: val_accuracy did not improve from 0.88736
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 165ms/step - accuracy: 0.9042 - loss: 0.2645 - val_accuracy: 0.8874 - val_loss: 0.2658 - learni

129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - accuracy: 0.9042 - loss: 0.2642 - val_accuracy: 0.8901 - val_loss: 0.2616 - learning_rate: 1.2500e-06
Epoch 118/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.9005 - loss: 0.2594
Epoch 118: val_accuracy did not improve from 0.89011
129/129 ━━━━━━━━━━━━━━━━━━━━ 41s 163ms/step - accuracy: 0.9004 - loss: 0.2594 - val_accuracy: 0.8874 - val_loss: 0.2595 - learning_rate: 1.2500e-06
Epoch 119/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.9052 - loss: 0.2655
Epoch 119: val_accuracy did not improve from 0.89011
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - accuracy: 0.9052 - loss: 0.2655 - val_accuracy: 0.8874 - val_loss: 0.2622 - learning_rate: 1.2500e-06
Epoch 120/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9115 - loss: 0.2601
Epoch 120: ReduceLROnPlateau reducing learning rate to 1e-06.

Epoch 120: val_accuracy did not improve from 0.89011
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 167ms/step - accuracy: 0.911

129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 170ms/step - accuracy: 0.9105 - loss: 0.2671 - val_accuracy: 0.8929 - val_loss: 0.2572 - learning_rate: 1.0000e-06
Epoch 133/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9106 - loss: 0.2534
Epoch 133: val_accuracy did not improve from 0.89286
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 168ms/step - accuracy: 0.9105 - loss: 0.2535 - val_accuracy: 0.8846 - val_loss: 0.2601 - learning_rate: 1.0000e-06
Epoch 134/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9152 - loss: 0.2538
Epoch 134: val_accuracy improved from 0.89286 to 0.89560, saving model to C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\models_cnn_traj\traj_cnn_20260109-023739.h5


129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 170ms/step - accuracy: 0.9150 - loss: 0.2540 - val_accuracy: 0.8956 - val_loss: 0.2583 - learning_rate: 1.0000e-06
Epoch 135/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.9044 - loss: 0.2626
Epoch 135: val_accuracy did not improve from 0.89560
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 168ms/step - accuracy: 0.9044 - loss: 0.2627 - val_accuracy: 0.8846 - val_loss: 0.2637 - learning_rate: 1.0000e-06
Epoch 136/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9184 - loss: 0.2457
Epoch 136: val_accuracy did not improve from 0.89560
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 167ms/step - accuracy: 0.9183 - loss: 0.2458 - val_accuracy: 0.8819 - val_loss: 0.2603 - learning_rate: 1.0000e-06
Epoch 137/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.9140 - loss: 0.2572
Epoch 137: val_accuracy did not improve from 0.89560
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 169ms/step - accuracy: 0.9139 - loss: 0.2573 - val_accuracy: 0.8819 - val_loss: 0.2658 - le

129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 166ms/step - accuracy: 0.9184 - loss: 0.2283 - val_accuracy: 0.8984 - val_loss: 0.2498 - learning_rate: 1.0000e-06
Epoch 185/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.9145 - loss: 0.2471
Epoch 185: val_accuracy did not improve from 0.89835
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 165ms/step - accuracy: 0.9144 - loss: 0.2472 - val_accuracy: 0.8901 - val_loss: 0.2515 - learning_rate: 1.0000e-06
Epoch 186/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.9134 - loss: 0.2424
Epoch 186: val_accuracy did not improve from 0.89835
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 163ms/step - accuracy: 0.9134 - loss: 0.2425 - val_accuracy: 0.8929 - val_loss: 0.2493 - learning_rate: 1.0000e-06
Epoch 187/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.9154 - loss: 0.2261
Epoch 187: val_accuracy did not improve from 0.89835
129/129 ━━━━━━━━━━━━━━━━━━━━ 22s 164ms/step - accuracy: 0.9154 - loss: 0.2263 - val_accuracy: 0.8874 - val_loss: 0.2463 - le